# Sampling and Law of Large Numbers
### Stephen Elston
### Data Science 350

## Introduction

Sampling is a fundamental process in the collection of data. Sampling is important because we can almost never look at the whole population. Some key points to keep in mind about sampling are:
- We use inferences on the sample to say something about the population.
- We need estimates of variances on the sample calculations to say something about the population.

Let's look at some examples of sampling.

| Use Case | Sample | Population |
|---|---|---|
| A/B Testing | The users we show either web sites A or B | All possible users, past present and future|
|World Cup Socer | 32 teams which qualify in one season | All national teams in past, present and future years|
|Average height of data science students | Students in UW Data Science 350 | All students taking data science classes world wide|

Notice, that in several cases it is not only impractical, but impossible to collect data from the entire population. Hence, we must work with correctly collected samples. 

## Sampling Strategies

We will investigate a number of commonly used sampling strategies: 

- Bernoulli Sampling
- Stratified Sampling
- Cluster Sampling
- Systematic Sampling


### Bernoulli Sampling

Bernoulli sampling has the following properties:

- Every point subjected to a probability of being selected. Simple Random Sample (most common)
- Fixed size Bernoulli sampling.
- Example randomly sample weight of product to ensure quality

Let's look at an example. The code in the cell below creates a data frame with 200 random samples of a Normal distribution, divided into 4 groups. Run this code and examine the summary of the sample. 

In [ ]:
# Create and data frame and look at the group size in the sample
data = data.frame(index = 1:200,
                  var1 = rnorm(200),
                  group = sample(1:4, 200, replace=TRUE, prob=c(0.1,0.3,0.4,0.2)))
require(dplyr)
data %>% group_by(group) %>% summarise(n.group = n(), mean.index = mean(var1))

Next, Bernoulli sample these data by runing the code in the cell below.  

In [ ]:
##----Bernoulli Sampling-----
p = 0.1
bernoulli_sample = data[runif(200)<p,] # Yay for R vectorization
bernoulli_sample %>% group_by(group) %>% summarise(n.group = n(), mean.index = mean(var1))

You can see that the total number of samples is approximately 10%. 

**Your Turn:** You can also use the R `sample` function to Bernoulli sample a data frame. Create the code to Bernoulli sample 15 cases from the population and print the summary statistcs of number and mean by group. 

Notice that for both samples the number of cases for some groups can be rather small. Compare the means by groups of the two samples and the population, and notice how much variation there is. Clearly, Bernoulli sampling is far from idea if the data have a group structure.  

### Stratified Sampling

Stratified sampling strategies are used when data are organized in groups or **Strata**. The idea is simple, sample each group in such a way that the sample includes a representative number of cases from each group. The simplist version of stratified sampling creates a sample with the same number of cases from each group.

Some examples of stratified sampling include:

- Sample equal numbers of men and women.
- Sample equal numbers of people in different income categories.
- Sample the same number of people from towns of different sizes.

Execute the code in the cell below to sample our population uniformly, and verify the result. 

In [ ]:
##-----Stratified Sampling for equal numbers -----
n = 5
stratified_sample = data %>% group_by(group) %>% sample_n(n, replace = FALSE)
stratified_sample %>% group_by(group) %>% summarise(n.group = n(), mean.index = mean(var1))

In [ ]:
stratified_sample %>% group_by(group) %>% summarise(n.group = n(), mean.index = mean(var1))

We now have 5 samples from each strata. Notice that the mean estimates of each group are closer those for the population. Stratified sampling ensures the samples for each group are more  representative of the groups. 

### Cluster Sampling

When sampling is expensive a strategy is required to reduce the cost, yet still keep randomized sampling. Some examples of data which is expensive to collect includues:

- Surveys of customers at a chain of stores.
- Door to door survey of homeowners.

In these cases, the popuation can be divided into clusters and a random selection of clusters sampled. The proceess of cluster sampling follows these steps:

- Define the clusters and divide the data.
- Randomly select some clusters.
- Sample from the selected clusters.
- Optionally, stratify the sample from the clusters.

As an example, you can select a few store locations and Bernoulli sample customers at these locations.

The code in the cell bellow divides a population into clusters, randomly selects 3 clusters and computes and prints some summary statistics. Run this code and examine the results.

In [ ]:
##----Cluster Sampling-----
num_clusters = 10
data$cluster_labels = sample(rep(1:num_clusters,each=2*num_clusters))
clusters_sampled = sample(1:10, 3) # Pick three clusters to sample
clusters_sampled
cluster_sample = data[data$cluster_labels %in% clusters_sampled,]
cluster_sample %>% group_by(cluster_labels) %>% summarise(n.group = n(), mean.index = mean(var1))

You can see the  3 clsuters of 20 cases each which have been selected.

Recalling that the population contians groups the sample from the clusters can be summarized by group by runing the code in the cell below. 

In [ ]:
cluster_sample %>% group_by(group) %>% summarise(n.group = n(), mean.index = mean(var1))

**Your Turn:** Sample the clusters by `group` and `cluster_labels`, compute the count and mean for the index, and then sort (`arrange`) by the  `cluster_labels` and `group`. Are the counts in the samples fairly different 

### Systematic Sampling

**WARNING: systmatic sampling is a form of convienience sampling. Convienience sampling almost always leads to problems!**

In systematic sampling every k th case of the population is selected. As you can imagine, this approach is not a random sampling method, but rather a case of convinience sampling. 

The code in the cell bellow systematically samples the population and computes some summary statistics. Run this code and examine the results. 

In [ ]:
##------Systematic Sampling-----
k = 5 # 100/5 = 20 observations
sys_sample_even = data[seq(1,nrow(data), by = k),]
sys_sample_even %>% group_by(group) %>% summarise(n.group = n(), mean.index = mean(var1))

Notice the differing sizes of the sample in each group and the bias in the mean estimates. This illustrates the hazards of convienience sampling. 

### A few more thoughts on sampling

- Whenever you are planing to sample data, make sure you have a clear sampling plan. 
- Know number of clusters, strata, samples in advance.
- Don’t stop sampling when desired result is achieved: e.g. error measure! 
- Note that random sampling, if done properly, controls for database effects, like indexing. 


## Law of large numers

The **law of large numbers** is a theorem that states that statistics of independent samples converge to the population values as more unbiased experiments are performed. We can write this mathematically for the **expected value** pr mean as:

$$Let\ \bar{X} = \frac{1}{n}\sum_{i=1}^{n} X_i\\
then\ by\ the\ law\ of\ Large\ Numbers\\
\bar{X} \rightarrow E(X) = \mu\\
as\\
n \rightarrow \infty$$

The law of large numbers is foundational to statistics. We rely on the law of large numbers whenever we work with samples of data. We can rest assured that larger samples will be more representatives of the population we are sampling. This theorem is the foundation of not only sampling theory, but modern compuation methods including, simulation, bootstrap resampling, and Monte Carlo methods. If the real world did not follow this theorem much of statistics, to say nothing of science and technology would fail badly. 

Given this great importance, it should not be surprising that the law of large numbers has a long history. Jacob Bernoulli postumously published the first proof for the Binomial distribution in 1713. In fact the law of large numbers is sometimes referred to as **Bernoulli's theorm**. A more general proof was published by Poisson in 1837.

Let's think about a simple example. The mean of 50 coin flips (0,1)=(T,H) is usually farther away from the true mean of 0.5 than 5,000 coin flips. The code in the cell below computes and then samples a population of 100,000,000 Bernoulli trials with $p=0.5$. Run this code and  examine the results. Does the expected value, or mean converge to 0.5 as `n` increases?

In [ ]:
pop = rbinom(100000000, 1, 0.5)
xs = c(5, 50, 500, 5000)
lapply(xs, function(x) mean(sample(pop, x)))

### Convergance of Bernolli trials 

Next, let's use the series of Bernuoulli trie to examine the convergance of the estimated probability, $p$, for a number of flips of a fair coin. This series should converge to $p = 0.5$ as the number of samples increses. The code in the cell below computes and plots (on two scales) the running mean of the realization of Bernoulli trials as the sample size increases. Run this code and examine the results. Do the results appear to be converging to $p$ as $n \rightarrow \infty$.

In [ ]:
# Calculate and plot a running average of N-trials of flipping a fair coin
n = 10000
running_average = sapply(1:n, function(x) mean(sample(pop, x)))
par(mfrow = c(2,1))      
plot(running_average[1:500], type='l')
grid()
plot(running_average, type='l')
grid()
par(mfrow = c(1,1))  

### Convergance for a Normal Distribution

As we sample from a normal distribution, the mean of the sample will converge to the population mean and the sample standard deviation will converge to the population statndard deviation. The standard error of the sample mean should converge as:

$$se = \frac{sd}{\sqrt(n)}$$

Run the code in the cell below to plot the convergance of the sample mean as the number of samples increases.

In [ ]:
##----St. Dev. vs. St. Error-----
n = seq(10,10000,len=1000)

sample_means = sapply(n, function(x) mean(rnorm(x)))
sample_sds = sapply(n, function(x) sd(rnorm(x)))

plot(n, sample_means) # Plot means
lines(n, 1/sqrt(n))   # Plot means +- st. error
lines(n, -1/sqrt(n))

**Your Turn:** In the cell below create and exectue code to plot the convergance of the sample standard deviation. Try to include lines showing the expected bounds on the standard error. 

##### Copyright 2017, Stephen F Elston. All rights reserved.